# Imports and utility functions

In [ ]:
!pip install datasets
!pip install transformers

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import transformers
from transformers import DistilBertTokenizerFast, BertTokenizerFast
from datasets import load_metric
import torch
import re

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 212 kB 37.1 MB/s 
     |████████████████████████████████| 1.1 MB 32.0 MB/s 
     |████████████████████████████████| 77 kB 2.9 MB/s 
     |████████████████████████████████| 136 kB 37.0 MB/s 
     |████████████████████████████████| 127 kB 36.2 MB/s 
     |████████████████████████████████| 271 kB 40.8 MB/s 
     |████████████████████████████████| 144 kB 52.2 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 5.1 MB/s 


In [ ]:
def cleanTexts(texts):
    cleaned = []
    pattern = "[^a-zA-Z0-9]"
    for text in texts:
        clrd = re.sub(pattern," ",text).lower().strip()
        cleaned.append(clrd)
    return cleaned

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Load data

In [ ]:
from sklearn.utils import shuffle

dataset = pd.read_csv('train_data_imdb.csv')
dataset = shuffle(dataset)
dataset.tail()

,Unnamed: 0,text,label
16153,5793,this movie is the best movie ever it has a lot...,1
3002,23794,What made me track this movie down was the vie...,0
13220,911,Give this movie a break! Its worth at least a ...,1
15709,17629,This ranks as one of the worst movies I've see...,0
15097,9041,Here's the kind of love story that I do enjoy ...,1


In [ ]:
x_train = list(cleanTexts(dataset['text']))
# print(x[:5])

y_train = list(dataset['label'])
# print(y[:5])

#x_train = x_train[:10000]
#y_train = y_train[:10000]

x_train = x_train[:25000]
y_train = y_train[:25000]

In [ ]:
test_data = pd.read_csv('test_data_imdb.csv')

x_test = list(cleanTexts(test_data['text']))
y_test = list(test_data['label'])

x_test = x_test[:10000]
y_test = y_test[:10000]

# Tokenize data

- **do not forget** to uncomment tokenizer for transformer you are using!

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, random_state=69)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
#tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class CreateDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CreateDataset(train_encodings, y_train)
val_dataset = CreateDataset(val_encodings, y_val)
test_dataset = CreateDataset(test_encodings, y_test)

# BERT

## 1.Experiment

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification



training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="steps",
    eval_steps = 500,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": 

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
creating metadata file for /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.458200,0.297603,0.901000,0.917471,0.876151,0.896335
1000,0.363700,0.476365,0.872000,0.816506,0.951894,0.879017
1500,0.256000,0.367114,0.897500,0.864151,0.937564,0.899362
2000,0.222600,0.333841,0.914500,0.903808,0.923234,0.913418
2500,0.095300,0.396308,0.920000,0.933192,0.900716,0.916667
3000,0.101500,0.408997,0.920500,0.915650,0.922211,0.918919


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin

TrainOutput(global_step=3000, training_loss=0.24954452133178712, metrics={'train_runtime': 5415.9319, 'train_samples_per_second': 4.431, 'train_steps_per_second': 0.554, 'total_flos': 6314665328640000.0, 'train_loss': 0.24954452133178712, 'epoch': 3.0})

In [ ]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 10000
  Batch size = 8


PredictionOutput(predictions=array([[ 1.8703463, -2.0145922],
       [-1.7720909,  2.1869917],
       [ 1.8777316, -2.0168655],
       ...,
       [ 2.0497708, -2.187519 ],
       [-1.5079627,  2.025078 ],
       [ 2.0310767, -2.1899588]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 1, 0]), metrics={'test_loss': 0.29256758093833923, 'test_accuracy': 0.8996, 'test_precision': 0.9137717121588089, 'test_recall': 0.882740711146624, 'test_f1': 0.8979882137776876, 'test_runtime': 693.1725, 'test_samples_per_second': 14.426, 'test_steps_per_second': 1.803})

## 2.Experiment

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification



training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="steps",
    eval_steps = 1000,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    #load_best_model_at_end=True,
)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,
)

trainer.train()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1000,0.367900,0.306259,0.898200,0.929734,0.862823,0.895030
2000,0.316000,0.347930,0.902600,0.857042,0.967793,0.909057
3000,0.211600,0.319004,0.922600,0.941128,0.902584,0.921453
4000,0.196600,0.328380,0.917800,0.954624,0.878330,0.914889
5000,0.139800,0.356919,0.927600,0.906994,0.953877,0.929845
6000,0.077600,0.342881,0.932600,0.934211,0.931610,0.932909
7000,0.051100,0.379638,0.932800,0.924425,0.943539,0.933884


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./

TrainOutput(global_step=7500, training_loss=0.20006452051798504, metrics={'train_runtime': 13548.7334, 'train_samples_per_second': 4.428, 'train_steps_per_second': 0.554, 'total_flos': 1.57866633216e+16, 'train_loss': 0.20006452051798504, 'epoch': 3.0})

In [ ]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 10000
  Batch size = 8


PredictionOutput(predictions=array([[ 3.1586113, -3.3124428],
       [-3.6524317,  3.4440715],
       [ 4.0087543, -3.9330769],
       ...,
       [ 4.135738 , -4.001612 ],
       [-4.197593 ,  3.8488598],
       [-4.1561975,  3.8407838]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 1, 1]), metrics={'test_loss': 0.3750956654548645, 'test_accuracy': 0.9338, 'test_precision': 0.9392655367231638, 'test_recall': 0.928030303030303, 'test_f1': 0.9336141195346971, 'test_runtime': 671.7992, 'test_samples_per_second': 14.885, 'test_steps_per_second': 1.861})

# DistilBERT

## 3.Experiment

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification



training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="steps",
    eval_steps = 500,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
)

model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 10000
  Batch size = 8


PredictionOutput(predictions=array([[ 1.8703463, -2.0145922],
       [-1.7720909,  2.1869917],
       [ 1.8777316, -2.0168655],
       ...,
       [ 2.0497708, -2.187519 ],
       [-1.5079627,  2.025078 ],
       [ 2.0310767, -2.1899588]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 1, 0]), metrics={'test_loss': 0.29256758093833923, 'test_accuracy': 0.8996, 'test_precision': 0.9137717121588089, 'test_recall': 0.882740711146624, 'test_f1': 0.8979882137776876, 'test_runtime': 693.1725, 'test_samples_per_second': 14.426, 'test_steps_per_second': 1.803})

## 4.Experiment

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification



training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="steps",
    eval_steps = 500,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
)

model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.predict(test_dataset)

# Save model

In [ ]:
torch.save(model, 'modelThird')